## 1. Import packages

In [ ]:
import tqdm
import random
import pathlib
import itertools
import collections

import cv2
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import keras
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import classification_report,accuracy_score

# Import the MoViNet model from TensorFlow Models (tf-models-official) for the MoViNet model
#from official.projects.movinet.modeling import w
#from official.projects.movinet.modeling import movinet_model

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import random
from pathlib import Path
import os
import cv2
import re
import collections
from sklearn.model_selection import train_test_split

kaggle_username = "rahaf8"
kaggle_key = "f59b8cb26f2973bc6fb4c52b1516ac19"
os.environ["KAGGLE_USERNAME"] = kaggle_username
os.environ["KAGGLE_KEY"] = kaggle_key
import kaggle

import warnings
warnings.filterwarnings('ignore')

## 2. Define Functions

#### 2.1 Get data from Kaggle

In [ ]:
def get_kaggle_dataset(dataset_name):
    """Get dataset from Kaggle.

    Args:
        dataset_name: the dataset name.
    """

    # Download the dataset using the Kaggle API
    kaggle.api.dataset_download_files(dataset_name, path=".", unzip=True)

#### 2.2 Video Dataframe Generator

In [ ]:
def video_dataframe(data_dir):
  """Get video dataframe.

    Args:
      files_path: A path from which the files can be stored.

    Returns:
      Video dataframe containing the labels , videos name , and videos path.
  """
  vidDf = pd.DataFrame(columns=['Label','VidName','VidPath'])

  for dirname, _, filenames in os.walk(data_dir):
      for name in filenames:
            vidDf =  vidDf.append({'Label': re.match(r'^[^\d_]+', name).group(),
                                   'VidName': name,
                                   'VidPath': os.path.join(dirname, name)},
                                    ignore_index=True)
  return vidDf

#### 2.3 Split Dataset

In [ ]:
def SplitData(testsize, df, classes):
    min_samples_per_class = min(df.groupby("Label").size())
    print(f"{min_samples_per_class} Samples per Class")

    df_TrainingSet = pd.DataFrame(columns=df.columns)
    df_TestSet = pd.DataFrame(columns=df.columns)

    for class_label in classes:
        df_class = df[df['Label'] == class_label].sample(min_samples_per_class, random_state=42)

        training_set, test_set = train_test_split(df_class, test_size=testsize, random_state=42)

        df_TrainingSet = df_TrainingSet.append(training_set)
        df_TestSet = df_TestSet.append(test_set)

    df_TrainingSet = df_TrainingSet.sample(frac=1, random_state=42)
    df_TestSet = df_TestSet.sample(frac=1, random_state=42)

    return df_TrainingSet, df_TestSet

#### 2.3 Move video into train and test

In [ ]:
import os
import shutil

def delete_empty_folders(directory):
    for root, dirs, files in os.walk(directory, topdown=False):
        for dir_name in dirs:
            folder_path = os.path.join(root, dir_name)
            delete_empty_folders(folder_path)  # Recursively check subdirectories
        if not os.listdir(root) and not files:
            os.rmdir(root)

def move_videos_to_folders(df, destination_dir):
    for _, row in df.iterrows():
        label = row['Label']
        vid_name = row['VidName']
        vid_path = row['VidPath']

        folder_path = os.path.join(destination_dir, label)
        os.makedirs(folder_path, exist_ok=True)

        new_vid_path = os.path.join(folder_path, vid_name)
        shutil.move(vid_path, new_vid_path)

        df.loc[df['VidPath'] == vid_path, 'VidPath'] = new_vid_path

#### 2.4 This code has been copied from tensorflow website without any changes - task for tomorrow

In [ ]:
def format_frames(frame, output_size):
  """
    Pad and resize an image from a video.

    Args:
      frame: Image that needs to resized and padded.
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of specified output size.
  """
  frame = tf.image.convert_image_dtype(frame, tf.float32)
  frame = tf.image.resize_with_pad(frame, *output_size)
  return frame

def frames_from_video_file(video_path, n_frames, output_size = (224,224), frame_step = 15):
  """
    Creates frames from each video file present for each category.

    Args:
      video_path: File path to the video.
      n_frames: Number of frames to be created per video file.
      output_size: Pixel size of the output frame image.

    Return:
      An NumPy array of frames in the shape of (n_frames, height, width, channels).
  """
  # Read each video frame by frame
  result = []
  src = cv2.VideoCapture(str(video_path))

  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

  need_length = 1 + (n_frames - 1) * frame_step

  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result

class FrameGenerator:
  def __init__(self, path, n_frames, training = False):
    """ Returns a set of frames with their associated label.

      Args:
        path: Video file paths.
        n_frames: Number of frames.
        training: Boolean to determine if training dataset is being created.
    """
    self.path = path
    self.n_frames = n_frames
    self.training = training
    self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
    self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

  def get_files_and_class_names(self):
    video_paths = list(self.path.glob('*/*.mp4'))
    classes = [p.parent.name for p in video_paths]
    return video_paths, classes

  def __call__(self):
    video_paths, classes = self.get_files_and_class_names()

    pairs = list(zip(video_paths, classes))

    if self.training:
      random.shuffle(pairs)

    for path, name in pairs:
      video_frames = frames_from_video_file(path, self.n_frames)
      label = self.class_ids_for_name[name] # Encode labels
      yield video_frames, label

## 2. Data Preprocessing

In [ ]:
dataset_name = "shashiprakash204/ucfcrimeminidataset"
dataset_dir = "/content/dataset/"

get_kaggle_dataset(dataset_name)

In [ ]:
video_df = video_dataframe(dataset_dir)
classes = video_df['Label'].unique().tolist()
print('Number of classes', len(classes))
print('Num videos for each class: : ')
print(video_df['Label'].value_counts())

In [ ]:
train_df, test_df = SplitData(0.2, video_df,classes)

In [ ]:
ddata = {"Training":train_df.groupby("Label").size(),"Test":test_df.groupby("Label").size()}

ddataframe = pd.DataFrame(data=ddata)
ddataframe.plot.bar(stacked= True, rot= 15, title='Training vs Test data',figsize=(15,5))
plt.show(block= True)

In [ ]:
train_destination_dir = '/content/dataset/train'
test_destination_dir = '/content/dataset/test'

# Move videos to train folders
move_videos_to_folders(train_df, train_destination_dir)

# Move videos to test folders
move_videos_to_folders(test_df, test_destination_dir)

#delete empty folders
delete_empty_folders('/content/dataset')

In [ ]:
batch_size = 8
num_frames = 8

output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))

train_ds = tf.data.Dataset.from_generator(FrameGenerator(Path(train_destination_dir), num_frames, training = True),
                                          output_signature = output_signature)
train_ds = train_ds.batch(batch_size)

test_ds = tf.data.Dataset.from_generator(FrameGenerator(Path(test_destination_dir), num_frames),
                                         output_signature = output_signature)
test_ds = test_ds.batch(batch_size)

In [ ]:
for frames, labels in train_ds.take(10):
  print(labels)

In [ ]:
print(f"Shape: {frames.shape}")
print(f"Label: {labels.shape}")

## the code below is copied from tensorflow website without any changes - task for tomorrow

In [ ]:
gru = layers.GRU(units=4, return_sequences=True, return_state=True)

inputs = tf.random.normal(shape=[1, 10, 8]) # (batch, sequence, channels)

result, state = gru(inputs) # Run it all at once

In [ ]:
first_half, state = gru(inputs[:, :5, :])   # run the first half, and capture the state
second_half, _ = gru(inputs[:,5:, :], initial_state=state)  # Use the state to continue where you left off.

print(np.allclose(result[:, :5,:], first_half))
print(np.allclose(result[:, 5:,:], second_half))

In [ ]:
model_id = 'a0'
resolution = 224

tf.keras.backend.clear_session()

backbone = movinet.Movinet(model_id=model_id)
backbone.trainable = False

# Set num_classes=600 to load the pre-trained weights from the original model
model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=600)
model.build([None, None, None, None, 3])

# Load pre-trained weights
!wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a0_base.tar.gz -O movinet_a0_base.tar.gz -q
!tar -xvf movinet_a0_base.tar.gz

checkpoint_dir = f'movinet_{model_id}_base'
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
status = checkpoint.restore(checkpoint_path)
status.assert_existing_objects_matched()

In [ ]:
def build_classifier(batch_size, num_frames, resolution, backbone, num_classes):
  """Builds a classifier on top of a backbone model."""
  model = movinet_model.MovinetClassifier(
      backbone=backbone,
      num_classes=num_classes)
  model.build([batch_size, num_frames, resolution, resolution, 3])

  return model

In [ ]:
model = build_classifier(batch_size, num_frames, resolution, backbone, 10)

In [ ]:
num_epochs = 2

loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

model.compile(loss=loss_obj, optimizer=optimizer, metrics=['accuracy'])

In [ ]:
results = model.fit(train_ds,
                    validation_data=test_ds,
                    epochs=num_epochs,
                    validation_freq=1,
                    verbose=1)

In [ ]:
model.evaluate(test_ds, return_dict=True)

In [ ]:
def get_actual_predicted_labels(dataset):
  """
    Create a list of actual ground truth values and the predictions from the model.

    Args:
      dataset: An iterable data structure, such as a TensorFlow Dataset, with features and labels.

    Return:
      Ground truth and predicted values for a particular dataset.
  """
  actual = [labels for _, labels in dataset.unbatch()]
  predicted = model.predict(dataset)

  actual = tf.stack(actual, axis=0)
  predicted = tf.concat(predicted, axis=0)
  predicted = tf.argmax(predicted, axis=1)

  return actual, predicted

In [ ]:
def plot_confusion_matrix(actual, predicted, labels, ds_type):
  cm = tf.math.confusion_matrix(actual, predicted)
  ax = sns.heatmap(cm, annot=True, fmt='g')
  sns.set(rc={'figure.figsize':(12, 12)})
  sns.set(font_scale=1.4)
  ax.set_title('Confusion matrix of action recognition for ' + ds_type)
  ax.set_xlabel('Predicted Action')
  ax.set_ylabel('Actual Action')
  plt.xticks(rotation=90)
  plt.yticks(rotation=0)
  ax.xaxis.set_ticklabels(labels)
  ax.yaxis.set_ticklabels(labels)

In [ ]:
fg = FrameGenerator(Path(train_destination_dir), num_frames, training = True)
label_names = list(fg.class_ids_for_name.keys())

In [ ]:
actual, predicted = get_actual_predicted_labels(test_ds)

In [ ]:
print(f'MobileNet Model accuracy on the test set is : {accuracy_score(actual, predicted )*100:.2f}%')